In [ ]:
# Image Sharpening Scratchpad (Python)

This notebook uses **Pillow (PIL)** to demonstrate image sharpening:

1. Environment / version check
2. Load `public/placeholder.jpg`
3. Create softened baseline (resize + blur)
4. Apply multiple sharpening variants
5. Summarize outputs and basic stats

Proceed to the next cell to set up the environment.

In [1]:
# Cell 2: Setup & version check
import sys, subprocess, importlib, os, platform

def ensure(pkg):
    try:
        return importlib.import_module(pkg)
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
        return importlib.import_module(pkg)

PIL = ensure('PIL')
from PIL import Image, ImageFilter, ImageEnhance, ImageStat

print('Python:', sys.version.split()[0])
print('Platform:', platform.platform())
print('Pillow version:', Image.__version__)
print('CWD:', os.getcwd())

Python: 3.9.6
Platform: macOS-15.6.1-arm64-arm-64bit
Pillow version: 11.1.0
CWD: /Users/animeshsareen/Desktop/code/photoshop-ai


In [ ]:
# Cell 3: Sharpening workflow
from pathlib import Path
from PIL import Image, ImageFilter, ImageEnhance, ImageStat
import math, json

base_dir = Path.cwd()
input_path = base_dir / 'public' / 'placeholder.jpg'
output_dir = base_dir / 'public' / 'sharp-tests-py'
output_dir.mkdir(parents=True, exist_ok=True)

if not input_path.exists():
    raise FileNotFoundError(f"Input image not found: {input_path}")

original = Image.open(input_path).convert('RGB')
print('Original size:', original.size)

# Create softened baseline (resize + gaussian blur)
soft = original.copy()
if soft.width > 512:
    soft.thumbnail((512, 512))
soft = soft.filter(ImageFilter.GaussianBlur(radius=1.2))
soft_path = output_dir / 'soft-source.jpg'
soft.save(soft_path, quality=92)

variants = []

def save_variant(img, name):
    out_path = output_dir / f'{name}.jpg'
    img.save(out_path, quality=92)
    kb = out_path.stat().st_size / 1024
    variants.append({'variant': name, 'kb': round(kb, 1)})

# 1. Built-in sharpen filter (mild)
sharpen_builtin = soft.filter(ImageFilter.SHARPEN)
save_variant(sharpen_builtin, 'sharpen_builtin')

# 2. More passes
multi_pass = soft.filter(ImageFilter.SHARPEN).filter(ImageFilter.SHARPEN)
save_variant(multi_pass, 'sharpen_multi_pass')

# 3. Unsharp Mask light
usm_light = soft.filter(ImageFilter.UnsharpMask(radius=1.5, percent=80, threshold=3))
save_variant(usm_light, 'unsharp_light')

# 4. Unsharp Mask strong
usm_strong = soft.filter(ImageFilter.UnsharpMask(radius=2.0, percent=180, threshold=2))
save_variant(usm_strong, 'unsharp_strong')

# 5. Contrast + mild sharpen combo
contrast_boost = ImageEnhance.Contrast(soft).enhance(1.1).filter(ImageFilter.UnsharpMask(radius=1.3, percent=120, threshold=4))
save_variant(contrast_boost, 'contrast_unsharp_combo')

print('Variants saved ->', output_dir)
print(json.dumps(variants, indent=2))

In [ ]:
# Cell 4: Basic sharpness metric (variance of Laplacian) & listing
import numpy as np
from pathlib import Path
from PIL import Image, ImageFilter
import cv2 as _cv2 if False else None  # placeholder to indicate optional cv2 (ignored)

try:
    import cv2
    have_cv = True
except ImportError:
    have_cv = False
    print('cv2 not installed: falling back to PIL edge filter variance proxy')

output_dir = Path.cwd() / 'public' / 'sharp-tests-py'
files = sorted([p for p in output_dir.glob('*.jpg')])

results = []
for f in files:
    img = Image.open(f).convert('L')
    arr = np.array(img)
    if have_cv:
        lap = cv2.Laplacian(arr, cv2.CV_64F)
        var_lap = float(lap.var())
    else:
        # Approximate: apply FIND_EDGES and compute variance
        edges = img.filter(ImageFilter.FIND_EDGES)
        var_lap = float(np.array(edges).var())
    results.append({'file': f.name, 'variance': round(var_lap, 2)})

# Sort descending by sharpness proxy
results.sort(key=lambda x: x['variance'], reverse=True)
for r in results:
    print(f"{r['file']:<28} variance={r['variance']}")

results